In [1]:
import functools
import os
os.environ['NCCL_DEBUG']='INFO'
#os.environ['TF_NCCL_VERSION']='2.13.4'
import collections
import contextlib
from jax.config import config
config.update("jax_enable_x64", True)
import jax
import jax.numpy as jnp
from jax import lax
from jax import random
import tensorflow as tf
tf.config.experimental.set_visible_devices([], "GPU")
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from tensorflow_probability.substrates.jax.internal.dtype_util
from tensorflow_probability.substrates import jax as tfp
import jax.profiler

tfd = tfp.distributions
tfb = tfp.bijectors
tfm = tfp.mcmc
tfed = tfp.experimental.distribute
tfde = tfp.experimental.distributions
tfem = tfp.experimental.mcmc

Root = tfed.JointDistributionCoroutine.Root
import pandas as pd
data = pd.read_pickle('/home/dkn16/data.pkl')
FGnopol = data.beam.foregrounds.all_nopol
FGpol = data.beam.foregrounds.pleak
HI = data.beam.HI
noise = data.beam.noise
freqs = data.freqs
#matplotlib.use("Agg")  # noqa: E402

2022-12-26 10:10:19.610425: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64:/work/dkn16/jax-env/lib:/home/dkn16/.local/lib/python3.7/site-packages:/usr/lib64:/work/dkn16/jax-env/lib:/home/dkn16/.local/lib/python3.7/site-packages:/opt/gsl/2.6/lib
2022-12-26 10:10:19.610804: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64:/work/dkn16/jax-env/lib:/home/dkn16/.local/lib/python3.7/site-packages:/usr/lib64:/work/dkn16/jax-env/lib:/home/dkn16/.local/lib/python3.7/site-packages:/opt/gsl/2.6/lib
2022-12-26 10:10:19.610810: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlo

In [2]:
print(jax.devices())

[StreamExecutorGpuDevice(id=0, process_index=0), StreamExecutorGpuDevice(id=1, process_index=0), StreamExecutorGpuDevice(id=2, process_index=0), StreamExecutorGpuDevice(id=3, process_index=0)]


In [3]:

# create artificial regression dataset
def get_data(dim,pol=False,x0=0,y0=0,freqs = 285):
    #idx=np.random.randint(1,6)
    #loc_idx=np.random.randint(48)
    
    #foreground
    if pol:
        sky = jnp.array(FGnopol+FGpol)[x0:x0+dim,y0:y0+dim,0:freqs].astype(jnp.float64)
    else:
        sky = jnp.array(FGnopol)[x0:x0+dim,y0:y0+dim,0:freqs].astype(jnp.float64)
    
    #HI signal
    cosmos = jnp.array(HI+noise)[x0:x0+dim,y0:y0+dim,0:freqs].astype(jnp.float64)
    
    sky = sky - jnp.mean(sky,axis=(0,1))
    cosmos = cosmos - jnp.mean(cosmos,axis=(0,1))
    
    sky=sky.reshape((dim*dim,freqs))/1000
    cosmos=cosmos.reshape((dim*dim,freqs))/1000
    
    np.random.seed(0)
    X = jnp.linspace(0., 1., sky.shape[1]).astype(jnp.float64)#,dtype = jnp.float64)
    #Y = X + 0.2 * jnp.power(X, 3.0) + 0.5 * jnp.power(0.5 + X, 2.0) * jnp.sin(4.0 * X)
    #Y += sigma_obs * np.random.randn(N)
    #Y -= jnp.mean(Y)
    #Y /= jnp.std(Y)

    #assert X.shape == (N,)
    #assert Y.shape == (N,)

    X_test = jnp.linspace(0., 1.,sky.shape[1]).astype(jnp.float64)#,dtype = jnp.float64)

    return X, sky+cosmos, X_test

In [11]:
def shard_value(x):
    x = x.reshape((jax.device_count(), -1, *x.shape[1:]))
    return jax.pmap(lambda x: x)(x) # pmap will physically place values on devices

shard = functools.partial(jax.tree_map, shard_value)
X, Y, X_test = get_data(256,pol=True,freqs=258)
Y=jnp.ones((260*260,258),dtype=jnp.float64)
Y = shard(Y)
Y.shape

(4, 16900, 258)

In [7]:
#This is 'one parameter for all'

from jax.experimental.host_callback import call
dtype = jnp.float64
#@jax.default_matmul_precision('float32')
@functools.partial(jax.pmap, axis_name='data', in_axes=(None, None,0), out_axes=(None,None))
def run(seed, X, data):
    #data = Y # a sharded dataset
    num_examples, dim = data.shape
    # this is our model 
    
    def model_fn():
        k= yield Root(tfd.Sample(tfd.HalfNormal(dtype(2.)), 1))
        k = k*jnp.eye(dim)
        #yield tfed.Sharded(tfd.MultivariateNormalFullCovariance(loc = jnp.zeros(dim,dtype=jnp.float64),covariance_matrix = k),
        #                   shard_axis_name='data')
        yield tfed.Sharded(tfd.Independent(tfd.MultivariateNormalFullCovariance(loc = jnp.zeros((num_examples,dim),dtype=jnp.float64),covariance_matrix = k),1),
                           shard_axis_name='data')
    model = tfed.JointDistributionCoroutine(model_fn)
    #print('yes')
    init_seed, sample_seed = random.split(seed)

    initial_state = model.sample(seed=init_seed)[:-1] # throw away `y`
    print(initial_state)

    def target_log_prob(*state):
        return model.log_prob((*state, data))
    print(target_log_prob(*initial_state))
    kernel = tfm.NoUTurnSampler(target_log_prob, 1e-2,max_tree_depth=8)
    kernel = tfm.DualAveragingStepSizeAdaptation(kernel,800,target_accept_prob=0.8)
    def trace_fn(state, pkr):
        log_prob = target_log_prob(*state)
        call(lambda x: print(f"step: {x[0]},leap_frogs: {x[1]},step_size: {x[2]:.2g},accept_ratio:{x[3]:.3f},log_prob:{x[4]:.2f} \n "), (pkr.step,pkr.inner_results.leapfrogs_taken,pkr.new_step_size,10**pkr.inner_results.log_accept_ratio,log_prob))
        #call(lambda x: print(f"leap_frogs: {x}"), pkr.inner_results.leapfrogs_taken)
        #call(lambda x: print(f"step_size: {x}"), pkr.new_step_size)
        
        return (
            log_prob,
            pkr.inner_results.has_divergence,
            10**pkr.inner_results.log_accept_ratio,
            #accuracy(*state),
            pkr.new_step_size,
        )
    states,traces = tfm.sample_chain(
            num_results=20,
            #num_burnin_steps=1000,
            current_state=initial_state,
            kernel=kernel,
            trace_fn=trace_fn,
            seed=sample_seed,
        )
    return states, traces

In [6]:
@functools.partial(jax.pmap, axis_name='data', in_axes=(None, None,0), out_axes=(0))
def run(seed, X, data):
    data=data*X
    
    
    return data

In [ ]:
%%time
states = run(random.PRNGKey(42),X,Y)
#states[0].block_until_ready()
#jax.profiler.save_device_memory_profile("memory.prof")
#jnp.save('states.npy',states)

StructTuple(
  var0=Traced<ShapedArray(float64[1])>with<DynamicJaxprTrace(level=0/1)>
)
Traced<ShapedArray(float64[])>with<DynamicJaxprTrace(level=0/1)>


2022-12-26 10:11:54.971361: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:752] failed to free device memory at 0x14695a34ac00; result: CUDA_ERROR_ILLEGAL_ADDRESS: an illegal memory access was encountered
2022-12-26 10:11:54.971389: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:752] failed to free device memory at 0x14695a2a3a00; result: CUDA_ERROR_ILLEGAL_ADDRESS: an illegal memory access was encountered
2022-12-26 10:11:54.971393: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:752] failed to free device memory at 0x14695a34ae00; result: CUDA_ERROR_ILLEGAL_ADDRESS: an illegal memory access was encountered
2022-12-26 10:11:54.971395: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:752] failed to free device memory at 0x14695a2a3c00; result: CUDA_ERROR_ILLEGAL_ADDRESS: an illegal memory access was encountered
2022-12-26 10:11:54.9713

AttributeError: 'CompiledFunction' object has no attribute 'shape'

In [ ]:
states = jnp.load('states.npy')
states[0][0]
# squared exponential kernel with diagonal noise term
def kernel(X, Z, var, length, noise, jitter=1.0e-12,is_noise=True):
    deltaXsq = jnp.power((X[:, None] - Z) / length, 2.0)
    k = 10*var * jnp.exp(-0.5 * deltaXsq)
    #k = var * jnp.exp(-0.5 * deltaXsq)
    if is_noise:
        k += (noise*noise*1.0e-10 + jitter) * jnp.eye(X.shape[0])
    return k

def predict(rng_key, X, Y, X_test, var, length, noise):
    # compute kernels between train and test data, etc.
    k_pp = kernel(X_test, X_test, var, length, noise,is_noise=False)
    k_pX = kernel(X_test, X, var, length, noise,is_noise=False)
    k_XX = kernel(X, X, var, length, noise,is_noise=True)
    K_xx_inv = jnp.linalg.inv(k_XX)
    K = k_pp - jnp.matmul(k_pX, jnp.matmul(K_xx_inv, jnp.transpose(k_pX)))
    #sigma_noise = jnp.sqrt(jnp.clip(jnp.diag(K), a_min=0.0)) * jax.random.normal(
    #    rng_key, X_test.shape[:1]
    #)
    mean = jnp.matmul(k_pX, jnp.matmul(K_xx_inv, Y.T)).T
    # we return both the mean function and a sample from the posterior predictive for the
    # given set of hyperparameters
    return mean#, mean + sigma_noise
vmap_args = (
    random.split(random.PRNGKey(3), 50),
    states[0][950:1000,0],
    states[2][950:1000,0],
    states[1][950:1000,0],
)
means = jax.vmap(
    lambda rng_key, var, length, noise: predict(
        rng_key, X, Y.reshape((-1,150)), X, var, length, noise
    )
)(*vmap_args)

mean_prediction = np.mean(means, axis=0)
percentiles = np.percentile(means, [5.0, 95.0], axis=0)

In [ ]:
Y1=Y.reshape((-1,150))
plt.fill_between(X, percentiles[0, 100,:], percentiles[1, 100,:], color="lightblue")
plt.scatter(X,Y1[100])
plt.plot(X,mean_prediction[100])


In [ ]:
cosmos=jnp.load(f'/work/dante/data/fg_rm_data/train_data/data_3/cosmos/HI_idx_1_loc_0.npy').astype(jnp.float64).transpose(2,1,0)
    
cosmos = cosmos - jnp.mean(cosmos,axis=(0,1))
cosmos = cosmos.reshape(256*256,150)

In [ ]:
i=13550
plt.plot(X,Y1[i]-mean_prediction[i])

plt.plot(X,cosmos[i]/10000)
plt.fill_between(X_test,Y1[i]-percentiles[0, i,:], Y1[i]-percentiles[1, i,:], color="lightblue")
plt.legend(['GPR','True'])

In [ ]:
i=20
plt.figure(figsize=(6,7),dpi=200)
plt.subplot(2,2,1)
plt.imshow(mean_prediction[:,i].reshape(256,256))
plt.title('recovered fg')
plt.subplot(2,2,2)
plt.imshow(Y1[:,i].reshape(256,256))
plt.title('true fg')
plt.subplot(2,2,3)
plt.imshow((Y1[:,i]-mean_prediction[:,i]).reshape(256,256)*10000,vmin=-0.5,vmax=1.4)
plt.title('recovered signal')
plt.colorbar()
plt.subplot(2,2,4)
plt.imshow(cosmos[:,i].reshape(256,256),vmin=-0.5,vmax=1.4)
plt.title('true signal')
plt.colorbar()

In [ ]:
k1 = kernel(X, X,  0.052016747, 0.23297304, 1.81647456)
k2 = kernel(X, X,  1.52016747, 0.23297304, 1.81647456)
k3 = kernel(X, X,  0.52016747, 0.23297304, 1.81647456)
k4 = kernel(X, X,  0.52016747, 0.03297304, 1.81647456)
rng_key, rng_key_predict = random.split(random.PRNGKey(4))
samp = numpyro.sample(
        "Y",
        dist.MultivariateNormal(loc=jnp.zeros((2,2,X.shape[0])), covariance_matrix=[[k1,k2],[k3,k4]]),
        #obs=Y,
        rng_key= rng_key,sample_shape=(8,)
    )
for i in range(8):
    plt.plot(X,samp[i][0][0]+15,c='tab:blue')
    plt.plot(X,samp[i][1][0]+5,c='tab:orange')
    plt.plot(X,samp[i][0][1]-5,c='tab:red')
    plt.plot(X,samp[i][1][1]-15,c='tab:green')

In [ ]:
numpyro.sample("kernel_var", dist.InverseGamma(10,2),rng_key= rng_key,sample_shape=(16,))

In [ ]:
percentiles.shape

In [ ]:
Y.shape

In [ ]:
k = kernel(X, X, 0.52016747, 0.23297304, 1.81647456)
jnp.mean(dist.MultivariateNormal(loc=jnp.zeros(X.shape[0]), covariance_matrix=k).log_prob(Y))

In [ ]:
samples

In [ ]:
import corner
data = np.vstack([samples['kernel_length'],samples['kernel_var'],samples['kernel_noise']]).T
corner.corner(data,labels=[
        r"$length\times 10$",
        r"$var$",
        r"$noise\times 10^{-5}$",
    ],quantiles=[0.16, 0.5, 0.84],show_titles=True)

In [ ]:
samp.shape

In [ ]:
Y.shape

gpu
